In [6]:
import pandas as pd
import numpy as np
import datetime
import os

In [1]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='Ludi24',
                         db='cp3_stockage')
# create cursor
cursor = connection.cursor()

In [3]:
data_sql = pd.read_sql_query("select * from table_data_stockage", connection)
data_sql

,id_enreg,id_serveur,name_serveur,pool,offre,capacity,used_capacity,date_enreg,origine,typologie,client
0,1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.83,2018-10-03,ibm,DAT,INF
1,2,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.83,2018-10-03,ibm,DAT,INF
2,3,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.46,2018-10-03,ibm,DAT,CMC
3,4,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.47,2018-10-03,ibm,DAT,CMC
4,5,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.53,2018-10-03,ibm,DAT,INF
...,...,...,...,...,...,...,...,...,...,...,...
561251,561252,79,SIGN_06DC3_T2S_SIGS_01_DS_02,mutapsan06,T2,5120.0,4196.75,2020-11-12,3par,VIR,SIG
561252,561253,73,SNIN_06T2S_ORASNI_BACKUP_01,mutapsan06,T2,45.0,42.94,2020-11-12,3par,DAT,SNI
561253,561254,69,SNIN_06T2S_ORASNI_DATA_FG2_01,mutapsan06,T2,200.0,169.06,2020-11-12,3par,DAT,SNI
561254,561255,70,SNIN_06T2S_ORASNI_DATA_FG2_02,mutapsan06,T2,200.0,170.05,2020-11-12,3par,DAT,SNI


In [4]:
table_fichiers_stockage_sql = pd.read_sql_query("select * from table_fichiers_stockage", connection)
table_fichiers_stockage_sql

,id_fichier,nom_fichier,plateforme
0,1,20181003_mutsapsan019_lssevdiskcopy.txt,IBM
1,2,20181004_mutsapsan019_lssevdiskcopy.txt,IBM
2,3,20181005_mutsapsan019_lssevdiskcopy.txt,IBM
3,4,20181006_mutsapsan019_lssevdiskcopy.txt,IBM
4,5,20181007_mutsapsan019_lssevdiskcopy.txt,IBM
...,...,...,...
2770,2771,20201110_mutapsan06_showvv-space.txt,3par
2771,2772,20201111_mutapsan05_showvv-space.txt,3par
2772,2773,20201111_mutapsan06_showvv-space.txt,3par
2773,2774,20201112_mutapsan05_showvv-space.txt,3par


# IMB

In [16]:
# création de la liste des fichiers à récupérer pour IBM volume
basepath_ibm_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/volume'
liste_ibm_volume=[]
with os.scandir(basepath_ibm_volume) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_ibm_volume.append(str(entry.name))
            
print(len(liste_ibm_volume))   

773
20201113_mutsapsan019_lssevdiskcopy.txt


In [15]:
fichiers_imb_a_ajouter = []
for i in liste_ibm_volume:
    if i not in table_fichiers_stockage_sql.nom_fichier.to_list():
        fichiers_imb_a_ajouter.append(i)
print(len(fichiers_imb_a_ajouter))

0


In [24]:
liste_test = [fichiers_imb_a_ajouter[-1]]
liste_test

['20201113_mutsapsan019_lssevdiskcopy.txt']

In [25]:
df_ibm_volume = pd.DataFrame()
for name in liste_test:
    with open(os.path.join(basepath_ibm_volume, name)) as f:
        df_temp = pd.read_fwf(f)
        df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
        df_ibm_volume = pd.concat([df_ibm_volume, df_temp])

df_ibm_volume = df_ibm_volume[df_ibm_volume.used_capacity.notnull()]
df_ibm_volume

,vdisk_id,vdisk_name,copy_id,mdisk_grp_id,mdisk_grp_name,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,grainsize,se_copy,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date
0,0,MUTN_19DC1_T0S_ORASIP_DATA_01,0,8,SDC1_T0,400.00GB,97.43GB,99.24GB,1.80GB,403,...,256.0,yes,no,97.43GB,8,SDC1_T0,yes,no,NaN,2020-11-13
1,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,0,6,SDC1_T2_FCM,200.00GB,199.80GB,200.01GB,217.25MB,99,...,256.0,yes,no,199.80GB,6,SDC1_T2_FCM,yes,no,NaN,2020-11-13
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,6,SDC1_T2_FCM,200.00GB,199.80GB,200.01GB,211.75MB,99,...,256.0,yes,no,199.80GB,6,SDC1_T2_FCM,yes,no,NaN,2020-11-13
3,3,MUTN_19DC3_T0S_ORASIP_DATA_01,0,1,SDC3_T0,400.00GB,97.44GB,99.24GB,1.80GB,403,...,256.0,yes,no,97.44GB,1,SDC3_T0,yes,no,NaN,2020-11-13
4,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,1,6,SDC1_T2_FCM,200.00GB,199.80GB,200.01GB,218.25MB,99,...,256.0,yes,no,199.80GB,6,SDC1_T2_FCM,yes,no,NaN,2020-11-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,753,PRVN_19DC1_T2M_PRVN_01_DS_01,1,3,SDC3_T2,512.00GB,86.44GB,96.68GB,10.24GB,529,...,NaN,no,yes,188.27GB,3,SDC3_T2,yes,no,NaN,2020-11-13
909,754,PRVN_19DC3_T2M_PRVN_01_DS_01,0,3,SDC3_T2,512.00GB,27.94GB,38.19GB,10.25GB,1340,...,NaN,no,yes,72.43GB,3,SDC3_T2,yes,no,NaN,2020-11-13
910,754,PRVN_19DC3_T2M_PRVN_01_DS_01,1,9,SDC1_T2,512.00GB,27.85GB,38.10GB,10.25GB,1343,...,NaN,no,yes,72.43GB,9,SDC1_T2,yes,no,NaN,2020-11-13
911,761,SIGN_19DC1_T2M_BACKUP_01,0,6,SDC1_T2_FCM,1.00TB,945.09GB,965.57GB,20.48GB,106,...,256.0,yes,no,945.09GB,6,SDC1_T2_FCM,yes,no,NaN,2020-11-13


In [26]:
# création d'une fonction qui recalcule les valeurs en MB et ne fait rien si la valeur null
import math
def en_MB(x):
    if type(x)==str:
        if "GB" in x :
            x = x.replace("GB","")
            x= float(x)*1024
        elif "G" in x:
            x = x.replace("G","")
            x= float(x)*1024
        elif "MB" in x:
            x = x.replace("MB","")
            x= float(x)
        elif "TB" in x:
            x = x.replace("TB","")
            x = float(x)*1024*1024
    elif math.isnan(float(x)):
        pass
    return x
def en_giga(x):
    return x/1024

# Fonction qui trouve la typologie du server
def typo(serveur):
    if "APP" in serveur:
        typo = "APP"
    elif "WEB" in serveur:
        typo = "WEB"
    elif "DAT" in serveur:
        typo = "DAT"
    elif "SQL" in serveur:
        typo = "DAT"
    elif "ORA" in serveur:
        typo = "DAT"
    elif "VIR" in serveur:
        typo = "VIR"
    elif "vir" in serveur:
        typo = "VIR"
    elif "FIC" in serveur:
        typo = "FIC"
    elif "MES" in serveur:
        typo = "MES"
    elif "DNS" in serveur:
        typo = "DNS"
    else:
        typo = "AUTRE"
    return typo

In [27]:
# transformation des colonnes capacity, used_capacity,real_capacity, free_capacity, uncompressed_used_capacity, used_capacity_before_reduction
for colonne in ["capacity", "used_capacity","real_capacity", "free_capacity", "uncompressed_used_capacity", "used_capacity_before_reduction"]:
    df_ibm_volume[colonne] = df_ibm_volume[colonne].map(lambda x : en_MB(x) )
# renommage des colonnes
df_ibm_volume = df_ibm_volume.rename(columns={"vdisk_id":"ID","vdisk_name":"name","mdisk_grp_name": "pool"})

#création d'une colonne offre
df_ibm_volume['offre'] = df_ibm_volume.pool
df_ibm_volume.offre= df_ibm_volume.offre.map(lambda x: x[5:])

# création d'une colonne origine
df_ibm_volume["origine"] = np.array("ibm")
df_ibm_volume_reduit = df_ibm_volume[["ID","name","pool","offre","capacity","used_capacity",'date','origine']].copy()

# réinitialise les index
df_ibm_volume_reduit.reset_index(drop=True,inplace =True)
df_ibm_volume_GB = df_ibm_volume_reduit.copy()
for colonne in ["capacity", "used_capacity"]:
     df_ibm_volume_GB[colonne] = df_ibm_volume_reduit[colonne].map(lambda x : en_giga(x)).copy()
df_ibm_volume_GB

,ID,name,pool,offre,capacity,used_capacity,date,origine
0,0,MUTN_19DC1_T0S_ORASIP_DATA_01,SDC1_T0,T0,400.0,97.43,2020-11-13,ibm
1,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2_FCM,T2_FCM,200.0,199.80,2020-11-13,ibm
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2_FCM,T2_FCM,200.0,199.80,2020-11-13,ibm
3,3,MUTN_19DC3_T0S_ORASIP_DATA_01,SDC3_T0,T0,400.0,97.44,2020-11-13,ibm
4,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,SDC1_T2_FCM,T2_FCM,200.0,199.80,2020-11-13,ibm
...,...,...,...,...,...,...,...,...
908,753,PRVN_19DC1_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,86.44,2020-11-13,ibm
909,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,27.94,2020-11-13,ibm
910,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC1_T2,T2,512.0,27.85,2020-11-13,ibm
911,761,SIGN_19DC1_T2M_BACKUP_01,SDC1_T2_FCM,T2_FCM,1024.0,945.09,2020-11-13,ibm


In [28]:
# création du DF Server volmue avec un fichier server IBM (le plus récent) 
# taille du fichier 175 000 ko.
basepath ='C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/server'
df_ibm_server = pd.DataFrame()
for name in ["20200930_mutsapsan019_lshostvdiskmap.txt"]:
    with open(os.path.join(basepath, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID', 'name',"SCSI_id" ,"vdisk_id" ,"vdisk_name" ,"vdisk_UID","IO_group_id","IO_group_name", "mapping_type","host_cluster_id","host_cluster_name"])
        df_temp['date']= name[:8]
        df_ibm_server = pd.concat([df_ibm_server, df_temp])


# supprime les lignes où le name n'est pas bon.
df_ibm_server = df_ibm_server.drop(df_ibm_server.name[df_ibm_server.name == "name"].index)
df_ibm_server = df_ibm_server.drop(df_ibm_server.name[df_ibm_server.vdisk_name.isnull()].index)

# création d'une colonne typo
df_ibm_server['typologie'] = df_ibm_server.name
df_ibm_server.typologie = df_ibm_server.typologie.map(lambda x : typo(x))

In [29]:
#Création d'un df pour connaitre la typologie du volume et le nom de client(si possible) à partir d'un nom de server associé
liste_volume = []
liste_type = []
liste_client =[]
for volume in df_ibm_server.vdisk_name.unique():
    if volume in liste_volume:
        pass
    else:
        liste_volume.append(volume)
        liste_client.append(df_ibm_server[df_ibm_server.vdisk_name == volume].name.unique()[0][:3])
        liste_type.append(df_ibm_server[df_ibm_server.vdisk_name == volume].typologie.unique()[0])
df_t = pd.DataFrame({"name_v":liste_volume,"client":liste_client,"type":liste_type})

In [30]:
# Etiquetage en typologie et nom des clients des volumes ibm à partir du dataframe constitué précédemment df_t.
# Si le serveur n'est pas reconnu, on tente de trouver la typo via son nom avec la fonction typo()
df_ibm_volume_plus = df_ibm_volume_GB.copy()

liste_typo =[]
liste_client=[]
for v in df_ibm_volume_plus.name:
    if v in df_t.name_v.unique().tolist(): # récupération de la typologie via le df df_t
        liste_typo.append(df_t[df_t.name_v == v].type.values[0])
        liste_client.append(df_t[df_t.name_v == v].client.values[0])
    else:
        if v[:3]=="INF":
            liste_typo.append(typo(v))   # appel de la fonction typo() pour trouver la typologie
            liste_client.append('INF')
        elif v[:3]=="SIG":
            liste_typo.append(typo(v))   
            liste_client.append('SIG')
        else :
            liste_typo.append(typo(v)) 
            liste_client.append('Client_non_enregistré')
df_ibm_volume_plus['typologie'] = liste_typo
df_ibm_volume_plus['client'] = liste_client
df_ibm_volume_plus

,ID,name,pool,offre,capacity,used_capacity,date,origine,typologie,client
0,0,MUTN_19DC1_T0S_ORASIP_DATA_01,SDC1_T0,T0,400.0,97.43,2020-11-13,ibm,DAT,MUT
1,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2_FCM,T2_FCM,200.0,199.80,2020-11-13,ibm,DAT,CMC
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2_FCM,T2_FCM,200.0,199.80,2020-11-13,ibm,DAT,CMC
3,3,MUTN_19DC3_T0S_ORASIP_DATA_01,SDC3_T0,T0,400.0,97.44,2020-11-13,ibm,DAT,MUT
4,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,SDC1_T2_FCM,T2_FCM,200.0,199.80,2020-11-13,ibm,DAT,CMC
...,...,...,...,...,...,...,...,...,...,...
908,753,PRVN_19DC1_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,86.44,2020-11-13,ibm,AUTRE,Client_non_enregistré
909,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,27.94,2020-11-13,ibm,AUTRE,Client_non_enregistré
910,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC1_T2,T2,512.0,27.85,2020-11-13,ibm,AUTRE,Client_non_enregistré
911,761,SIGN_19DC1_T2M_BACKUP_01,SDC1_T2_FCM,T2_FCM,1024.0,945.09,2020-11-13,ibm,AUTRE,SIG


In [ ]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in df_ibm_volume_plus.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_ibm_volume_plus.iterrows():
    sql = "INSERT INTO `table_data_stockage` (`" + cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

# 3par

In [31]:
# création de la liste des fichiers à récupérer pour 3par volume

basepath_3par_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/3par/volume'
# List all files in a directory using scandir()
liste_3par_volume=[]
with os.scandir(basepath_3par_volume) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_3par_volume.append(str(entry.name))
print(len(liste_3par_volume))

2002


In [32]:
fichiers_3par_a_ajouter = []
for i in liste_3par_volume:
    if i not in table_fichiers_stockage_sql.nom_fichier.to_list():
        fichiers_3par_a_ajouter.append(i)
print(len(fichiers_3par_a_ajouter))

0


In [ ]:
# création du DF principal à partir de la liste_3par_volume

df_3par_volume = pd.DataFrame()
for name in liste_3par_volume:
    with open(os.path.join(basepath_3par_volume, name)) as f:
        
        # gestion des formats de fichiers avant / après le 08 janvier 2020
        if datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')<= datetime.datetime(2020,1,8):
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","VSize","Compaction","Dedup"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
            df_temp = df_temp.rename(columns={"Tot_Rsvd": "Tot_used"})
        else:
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","Tot_used","HostWR","VSize","Compact","Compress"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
        df_3par_volume = pd.concat([df_3par_volume, df_temp])
        
# supprime les lignes où le name n'est pas bon.
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name.isnull()].index)
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name=='total'].index)
df_3par_volume = df_3par_volume[df_3par_volume.name !="No"]
# renommage des colonnes
df_3par_volume = df_3par_volume.rename(columns={"Tot_used": "used_capacity", "VSize":"capacity"})

# limitation du nb de colonnes
df_3par_volume = df_3par_volume[["ID","name","capacity","used_capacity","date","pool"]]
df_3par_volume['offre'] = np.array('T2')
#suppression des lignes où la used_capcity est null
df_3par_volume = df_3par_volume[df_3par_volume.used_capacity.notnull()]

# réinitialise les index
df_3par_volume.reset_index(drop=True,inplace =True)

# traitement sur la colonne date
df_3par_volume.date = df_3par_volume.date.map(lambda x :  datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d'))

# ajout d'une colonne origine
df_3par_volume['origine'] = np.array("3par")
df_3par_volume

In [ ]:
# ajout de colonne Client
df_3par_volume['client'] = df_3par_volume.name
df_3par_volume.client = df_3par_volume.client.map(lambda x : x[:3])
df_3par_volume

In [ ]:
# conversion en GB
df_3par_volume_GB = df_3par_volume.copy()
for colonne in ['capacity','used_capacity']:
    df_3par_volume_GB[colonne] = df_3par_volume_GB[colonne].map(lambda x :en_giga(x))
df_3par_volume_GB

In [ ]:
# création du DF Server volume avec un fichier server 3par (le plus récent 20191027) 

basepath_3par_server = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/3par/server/lun'
df_3par_server = pd.DataFrame()
for name in ["20191027_mutapsan06_showvlun-lvw.txt"]:
    with open(os.path.join(basepath_3par_server, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID_volume', 'name_volume',"vv_wwn" ,"server" ,"Host_WWN/ISCSI" ,"port","Type","Status", "id"])
        df_temp['date']= name[:8]
        df_3par_server = pd.concat([df_3par_server, df_temp])
        
# supprime les lignes où le name n'est pas bon.
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VVName"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VLUNs"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.ID_volume == "VLUN"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "total"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server['Host_WWN/ISCSI'] == "----------------"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume.isnull()].index)
df_3par_server

In [ ]:
df_3par_server['typologie'] = df_3par_server.server
df_3par_server.typologie = df_3par_server.typologie.map(lambda x : typo(x))

In [ ]:
# étiquetage de la typologie des volumes 3par
df_3par_volume_plus = df_3par_volume_GB.copy()
liste_typo =[]
for v in df_3par_volume_plus.name:
    if v in df_3par_server.name_volume.unique().tolist():
        liste_typo.append(df_3par_server[df_3par_server.name_volume == v].typologie.values[0])
    else:
        liste_typo.append(typo(v))
        
df_3par_volume_plus['typologie'] = liste_typo
df_3par_volume_plus

In [ ]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in df_3par_volume_plus.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_3par_volume_plus.iterrows():
    sql = "INSERT INTO `table_data_stockage` (`" + cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
df_fichiers_stockage_ibm = pd.DataFrame({'nom_fichier':fichiers_imb_a_ajouter,'plateforme':np.array('IBM')})
df_fichiers_stockage_3par = pd.DataFrame({'nom_fichier':fichiers_3par_a_ajouter,'plateforme':np.array('3par')})
df_fichiers_stockage = pd.concat([df_fichiers_stockage_ibm,df_fichiers_stockage_3par])
df_fichiers_stockage

In [ ]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in df_fichiers_stockage.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_fichiers_stockage.iterrows():
    sql = "INSERT INTO `table_fichiers_stockage` (`" + cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"

    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
connection.close()